In [1]:
import cv2
import numpy as np

In [2]:
img = cv2.imread('D:/test_pic/1.jpg',1)
cv2.imshow('original',img)
cv2.waitKey(0)

-1

In [3]:
def pinyi_warp_1(img ,tx, ty):#tx,ty-->平移距离
    h, w, c= img.shape
    M=np.float32([[1,0,tx],[0,1,ty]])
    new_img = np.zeros_like(img)
    for y in range(h):
        for x in range(w):
            src_pos = np.array([x,y,1])
            dst_pos = np.dot(M,src_pos)
            dst_x,dst_y = int(dst_pos[0]),int(dst_pos[1])
            if dst_x<0 or dst_x>=w or dst_y <0 or dst_y>=h:
                continue
            new_img[dst_y,dst_x] = img[y,x]
    return new_img

In [4]:
new_img = pinyi_warp_1(img,130,-80)
cv2.imshow('new',new_img)
cv2.waitKey(0)
#cv2.destroyAllWindows()

-1

In [5]:
import math
def rotate_warp_1(img,angle):#angle直接输入想要的角度值即可，不必考虑弧度值
    h, w, c= img.shape
    center = (w//2,h//2)
    angle = math.radians(angle)
    cos_angle = math.cos(angle)
    sin_angle = math.sin(angle)
    M = np.array([[cos_angle,-sin_angle,0],[sin_angle,cos_angle,0],[0,0,1]])
    #调整平移部分
    t = np.array([[1,0,center[0]],[0,1,center[1]],[0,0,1]])
    t_inv = np.linalg.inv(t)
    M = np.dot(np.dot(t,M),t_inv)
    new_w = int((h*sin_angle)+(w*cos_angle))
    new_h = int((h*cos_angle)+(w*sin_angle))
    #调整旋转矩阵中的平移部分
    M[0,2]+=new_w/2 - center[0]
    M[1,2]+=new_h/2-center[1]
    new_img = np.zeros((new_h,new_w,img.shape[2]),dtype = np.uint8)
    for y in range(h):
        for x in range(w):
            src_pos = np.array([x,y,1])
            dst_pos = np.dot(M,src_pos)
            dst_x,dst_y = int(dst_pos[0]),int(dst_pos[1])
            if dst_x<0 or dst_x>=new_w or dst_y <0 or dst_y>=new_h:
                continue
            new_img[dst_y,dst_x] = img[y,x]
    return new_img

In [6]:
new_img = rotate_warp_1(img,45)
cv2.imshow('new',new_img)
cv2.waitKey(0)

-1

In [7]:
#滤一下，去除黑点
new_img_m = cv2.medianBlur(new_img,3)
cv2.imshow('new',new_img_m)
cv2.waitKey(0)

-1

In [8]:
import math
def resize_image_1(img,scale_p):
    h, w, c= img.shape
    new_w = int(w*scale_p/100)
    new_h = int(h*scale_p/100)
    new_imag = np.zeros((new_h,new_w,3),np.uint8)
    #计算缩放比例
    x_scale = float(img.shape[1]-1)/(new_w-1)
    y_scale = float(img.shape[0]-1)/(new_h-1)
    #双线性插值
    for i in range(new_h):
        for j in range(new_w):
            x=j*x_scale
            y=i*y_scale
            x1=int(x)
            y1=int(y)
            x2=min(x1+1,img.shape[1]-1)
            y2=min(y1+1,img.shape[0]-1)
            dx1=x-x1
            dy1=y-y1
            dx2=1-dx1
            dy2=1-dy1
            b1,g1,r1=img[y1,x1]
            b2,g2,r2=img[y1,x2]
            b3,g3,r3=img[y2,x1]
            b4,g4,r4=img[y2,x2]
            blue = int(b1*dx2*dy2+b2*dx1*dy2+b3*dx2*dy1+b4*dx1*dy1)
            green = int(g1*dx2*dy2+g2*dx1*dy2+g3*dx2*dy1+g4*dx1*dy1)
            red = int(r1*dx2*dy2+r2*dx1*dy2+r3*dx2*dy1+r4*dx1*dy1)
            new_imag[i,j] = [blue,green,red]
    return new_imag
def resize_image_2(img,scale_p):
    h, w, c= img.shape
    #直接转换不太行，改成了向上取整会好很多
    new_w = math.ceil(w*scale_p/100)
    new_h = math.ceil(h*scale_p/100)
    Sx=Sy=scale_p/100
    M=np.float32([[Sx,0,0],[0,Sy,0]])
    new_imag = np.zeros((new_h,new_w,3),np.uint8)
    for y in range(h):
        for x in range(w):
            o_p = np.array([x,y,1])
            new_point = M.dot(o_p)
            new_x,new_y = int(new_point[0]),int(new_point[1])
            if new_x<img.shape[1] and new_y <img.shape[0]:
                new_imag[new_y,new_x] = img[y,x]
    return new_imag

In [9]:
new_imge = resize_image_2(img,60)
cv2.imshow('new',new_imge)
cv2.waitKey(0)

-1

In [10]:
def shear_img(img,theta1,theta2):
    h, w, c= img.shape
    t1 = math.radians(theta1)
    t2 = math.radians(theta2)
    shx = math.tan(t1)
    shy = math.tan(t2)
    #矩阵
    M = np.float32([[1,shx,0],[shy,1,0],[0,0,1]])
    new_w = int(w+abs(shx)*h)
    new_h = int(h+abs(shy)*w)
    new_img = np.zeros((new_h,new_w,3),dtype = np.uint8)
    for y in range(h):
        for x in range(w):
            o_p=np.array([x,y,1])
            dst_p = M.dot(o_p)
            new_x,new_y = int(dst_p[0]),int(dst_p[1])
            if 0<=new_x<new_w and 0<=new_y<new_h:
                new_img[new_y,new_x] = img[y,x]
    return new_img

In [11]:
new_imge = shear_img(img,30,15)
cv2.imshow('new',new_imge)
cv2.waitKey(0)

-1

In [12]:
def perspect_img(img,src,dst):
    h, w, c= img.shape
    #计算透视变换矩阵
    A = np.zeros((8,9))
    for i in range(4):
        x,y = src[i]
        u,v = dst[i]
        A[2*i] = [-x,-y,-1,0,0,0,u*x,u*y,u]
        A[2*i+1] = [0,0,0,-x,-y,-1,v*x,v*y,v]
    U, S, V = np.linalg.svd(A)
    H = V[-1,:].reshape((3,3))
    new_img = np.zeros((h,w,3),dtype = np.uint8)
    for y in range(h):
        for x in range(w):
            o_p = np.array([x,y,1])
            new_p = H.dot(o_p)
            new_x,new_y,z= new_p/new_p[2]
            if 0<=new_x<w and 0<=new_y<h:
                new_img[int(new_y),int(new_x)] = img[y,x]
    return new_img

In [ ]:
#四个点从左上后=的点开始，顺时针旋转
src_points = np.float32([[200,450],[550,450],[450,200],[200,200]])
dst_points = np.float32([[300,200],[500,200],[500,100],[300,100]])
new_img = perspect_img(img,src_points,dst_points)
cv2.imshow('new',new_img)
cv2.waitKey(0)